In [1]:
import pandas as pd
import requests
import os
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
#https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95
# Motor Vehicle Collisions - Crashes
#https://dev.socrata.com/foundry/data.cityofnewyork.us/h9gi-nx95

In [4]:
endpoint = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json'
response = requests.get(endpoint)
data = response.json()
df = pd.json_normalize(data)

In [5]:
df.drop_duplicates()

,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,cross_street_name,location.latitude,location.longitude,location.human_address,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2021-09-11T00:00:00.000,2:39,WHITESTONE EXPRESSWAY,20 AVENUE,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,QUEENSBORO BRIDGE UPPER,NaN,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-29T00:00:00.000,6:55,THROGS NECK BRIDGE,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-11T00:00:00.000,9:35,NaN,NaN,0,0,0,0,0,0,...,1211 LORING AVENUE,40.667202,-73.8665,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
4,2021-12-14T00:00:00.000,8:13,SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,...,NaN,40.683304,-73.917274,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021-04-14T00:00:00.000,12:47,HENDRIX STREET,ATLANTIC AVENUE,2,0,0,0,0,0,...,NaN,40.676594,-73.89038,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
996,2021-04-16T00:00:00.000,14:30,EAST 64 STREET,NaN,0,0,0,0,0,0,...,NaN,40.76468,-73.9643,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
997,2021-04-15T00:00:00.000,0:00,WEST 155 STREET,BROADWAY,3,0,0,0,0,0,...,NaN,40.832764,-73.94583,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN
998,2021-04-14T00:00:00.000,6:55,BROOKLYN QUEENS EXPRESSWAY,NaN,0,0,0,0,0,0,...,NaN,40.698544,-73.96236,"{""address"": """", ""city"": """", ""state"": """", ""zip""...",NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Selecting less columns and changind data types

df = df.iloc[:,[0,4,5]]
df['crash_date'] = pd.to_datetime(df['crash_date'])

#df['number_of_persons_injured'] = df['number_of_persons_injured'].values.astype(int)
#df['number_of_persons_killed']= df['number_of_persons_killed'].values.astype(int)

df['number_of_persons_injured'] =  pd.to_numeric(df['number_of_persons_injured'], errors="coerce").fillna(0).astype('int64')
df['number_of_persons_killed'] =  pd.to_numeric(df['number_of_persons_killed'], errors="coerce").fillna(0).astype('int64')

In [7]:
#Group data by day
df = df.groupby(by='crash_date').sum()

In [8]:
df= df.reset_index()

In [9]:
df.head()

,crash_date,number_of_persons_injured,number_of_persons_killed
0,2016-04-16,0,0
1,2019-05-21,0,0
2,2020-01-21,0,0
3,2020-04-15,0,0
4,2021-02-26,0,0


In [10]:
df.dtypes

crash_date                   datetime64[ns]
number_of_persons_injured             int64
number_of_persons_killed              int64
dtype: object

In [11]:
df['crash_date'] = df['crash_date'].astype(str)

In [12]:
#Crear tabla en Amazon Redshift

In [13]:
#Connect to the cluster
import redshift_connector

In [14]:
conn = redshift_connector.connect(
     host=os.getenv('host'),
     database=os.getenv('database'),
     port=5439,
     user=os.getenv('user'),
     password=os.getenv('password'),
  )

In [15]:
# Create a Cursor object
cursor = conn.cursor()

In [16]:
#Create Table
create_table_sql = "CREATE TABLE guidogiantorno_coderhouse.collisions4 (crash_date varchar,number_of_persons_injured int,number_of_persons_killed int) DISTKEY(crash_date) SORTKEY(crash_date);"

cursor.execute(create_table_sql)

In [17]:
# Delete everything from the table to avoid inserting multiple times
cursor.execute("delete from guidogiantorno_coderhouse.collisions4")

In [21]:
#Insert Values

# Define the SQL statement to insert data into the table
insert_sql = """
    INSERT INTO guidogiantorno_coderhouse.collisions4 (crash_date, number_of_persons_injured, number_of_persons_killed)
    VALUES (%s, %s, %s)
"""

# Generate the values to be inserted as a list of tuples
values = [tuple(row) for row in df.values]

# Execute the SQL statement to insert data into the table
cursor.executemany(insert_sql, values)


In [22]:
# Commit the changes to make them permanent
conn.commit()

In [23]:
# Query a table using the Cursor
cursor.execute("select * from guidogiantorno_coderhouse.collisions4")